In [1]:
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[2]").appName('kmeans').enableHiveSupport().getOrCreate()
conf = SparkConf().setAppName("kmeansExample").setMaster("local[2]")
sc = spark.sparkContext
from pyspark.ml.feature import *

In [11]:
#通过设置阈值，二分类特征
df = spark.createDataFrame([(0.5,)], ["values"])
binarizer = Binarizer(threshold=1.0, inputCol="values", outputCol="features")
binarizer.transform(df).show()
binarizer.setParams(outputCol='freqs').transform(df).show()
binarizer.setOutputCol('freqs').transform(df).show()
binarizer.getThreshold()
# binarizerPath = temp_path + "/binarizer"
# binarizer.save(binarizerPath)
# loadedBinarizer = Binarizer.load(binarizerPath)

+------+--------+
|values|features|
+------+--------+
|   0.5|     0.0|
+------+--------+

+------+-----+
|values|freqs|
+------+-----+
|   0.5|  0.0|
+------+-----+

+------+-----+
|values|freqs|
+------+-----+
|   0.5|  0.0|
+------+-----+



1.0

In [35]:
#Bucketizer，离散化函数
values = [(0.1,), (0.4,), (1.2,), (1.5,), (float("nan"),), (float("nan"),)]
# print type(values[0])
df = spark.createDataFrame(values,["values"])
df.show()
bucketizer = Bucketizer(splits=[-float("inf"), 0.5, 1.4, float("inf")],inputCol="values",outputCol="buckets")
bucketed = bucketizer.setHandleInvalid("keep").transform(df)
bucketed.show()
# bucketizerPath = temp_path + "/bucketizer"
# bucketizer.save(bucketizerPath)
# loadedBucketizer = Bucketizer.load(bucketizerPath)
# loadedBucketizer.getSplits() == bucketizer.getSplits()
print bucketizer.getSplits()

+------+
|values|
+------+
|   0.1|
|   0.4|
|   1.2|
|   1.5|
|   NaN|
|   NaN|
+------+

+------+-------+
|values|buckets|
+------+-------+
|   0.1|    0.0|
|   0.4|    0.0|
|   1.2|    1.0|
|   1.5|    2.0|
|   NaN|    3.0|
|   NaN|    3.0|
+------+-------+

[-inf, 0.5, 1.4, inf]


In [41]:
#卡方选择器
# Chi-Squared feature selection, which selects categorical features to use for predicting a categorical label.
from pyspark.ml.linalg import Vectors
df = spark.createDataFrame(\
    [(Vectors.dense([0.0, 0.0, 18.0, 1.0]), 1.0),\
     (Vectors.dense([0.0, 1.0, 12.0, 0.0]), 0.0),\
     (Vectors.dense([1.0, 0.0, 15.0, 0.1]), 0.0)],\
    ["features", "label"])
selector = ChiSqSelector(numTopFeatures=2, outputCol="selectedFeatures")
model = selector.fit(df)
model.transform(df).show()
# chiSqSelectorPath = temp_path + "/chi-sq-selector"
# selector.save(chiSqSelectorPath)
# loadedSelector = ChiSqSelector.load(chiSqSelectorPath)
# loadedSelector.getNumTopFeatures() == selector.getNumTopFeatures()
# modelPath = temp_path + "/chi-sq-selector-model"
# model.save(modelPath)
# loadedModel = ChiSqSelectorModel.load(modelPath)
# loadedModel.selectedFeatures == model.selectedFeatures



+------------------+-----+----------------+
|          features|label|selectedFeatures|
+------------------+-----+----------------+
|[0.0,0.0,18.0,1.0]|  1.0|      [18.0,1.0]|
|[0.0,1.0,12.0,0.0]|  0.0|      [12.0,0.0]|
|[1.0,0.0,15.0,0.1]|  0.0|      [15.0,0.1]|
+------------------+-----+----------------+



In [45]:
# 词频统计
# class pyspark.ml.feature.CountVectorizer(self, minTF=1.0, minDF=1.0, vocabSize=1 << 18, binary=False, inputCol=None, outputCol=None)
# Extracts a vocabulary from document collections and generates a CountVectorizerModel.
df = spark.createDataFrame(\
     [(0, ["a", "b", "c"]), (1, ["ab", "b", "b", "c", "a"])],\
     ["label", "raw"])
cv = CountVectorizer(inputCol="raw", outputCol="vectors")
model = cv.fit(df)
model.transform(df).show(truncate=False)
sorted(model.vocabulary) == ['a','ab', 'b', 'c']
# countVectorizerPath = temp_path + "/count-vectorizer"
# cv.save(countVectorizerPath)
# loadedCv = CountVectorizer.load(countVectorizerPath)
# loadedCv.getMinDF() == cv.getMinDF()
# loadedCv.getMinTF() == cv.getMinTF()
# loadedCv.getVocabSize() == cv.getVocabSize()
# modelPath = temp_path + "/count-vectorizer-model"
# model.save(modelPath)
# loadedModel = CountVectorizerModel.load(modelPath)
# loadedModel.vocabulary == model.vocabulary

+-----+----------------+-------------------------------+
|label|raw             |vectors                        |
+-----+----------------+-------------------------------+
|0    |[a, b, c]       |(4,[0,1,2],[1.0,1.0,1.0])      |
|1    |[ab, b, b, c, a]|(4,[0,1,2,3],[2.0,1.0,1.0,1.0])|
+-----+----------------+-------------------------------+



True

In [47]:
#DCT 变换
# class pyspark.ml.feature.DCT(self, inverse=False, inputCol=None, outputCol=None)
# A feature transformer that takes the 1D discrete cosine transform of a real vector.
# No zero padding is performed on the input vector. It returns a real vector of the same length representing the DCT. 
# The return vector is scaled such that the transform matrix is unitary (aka scaled DCT-II).
from pyspark.ml.linalg import Vectors
df1 = spark.createDataFrame([(Vectors.dense([5.0, 8.0, 6.0]),)], ["vec"])
dct = DCT(inverse=False, inputCol="vec", outputCol="resultVec")
df2 = dct.transform(df1)
df2.head().resultVec
df3 = DCT(inverse=True, inputCol="resultVec", outputCol="origVec").transform(df2)
df3.head().origVec
# dctPath = temp_path + "/dct"
# dct.save(dctPath)
# loadedDtc = DCT.load(dctPath)
# loadedDtc.getInverse()

DenseVector([5.0, 8.0, 6.0])

In [49]:
#点乘
from pyspark.ml.linalg import Vectors
df = spark.createDataFrame([(Vectors.dense([2.0, 1.0, 3.0]),)], ["values"])
ep = ElementwiseProduct(scalingVec=Vectors.dense([1.0, 2.0, 3.0]),\
    inputCol="values", outputCol="eprod")
ep.transform(df).head().eprod
ep.setParams(scalingVec=Vectors.dense([2.0, 3.0, 5.0])).transform(df).head().eprod
# elementwiseProductPath = temp_path + "/elementwise-product"
# ep.save(elementwiseProductPath)
# loadedEp = ElementwiseProduct.load(elementwiseProductPath)
# loadedEp.getScalingVec() == ep.getScalingVec()


DenseVector([4.0, 3.0, 15.0])

In [50]:
#"TF是一种统计方法，用以评估一字词对于一个文件集或一个语料库中的其中一份文件的重要程度。
# 字词的重要性随着它在文件中出现的次数成正比增加，但同时会随着它在语料库中出现的频率成反比下降。
df = spark.createDataFrame([(["a", "b", "c"],)], ["words"])
hashingTF = HashingTF(numFeatures=10, inputCol="words", outputCol="features")
hashingTF.transform(df).head().features
hashingTF.setParams(outputCol="freqs").transform(df).head().freqs
params = {hashingTF.numFeatures: 5, hashingTF.outputCol: "vector"}
hashingTF.transform(df, params).head().vector
# hashingTFPath = temp_path + "/hashing-tf"
# hashingTF.save(hashingTFPath)
# loadedHashingTF = HashingTF.load(hashingTFPath)
# loadedHashingTF.getNumFeatures() == hashingTF.getNumFeatures()

SparseVector(5, {0: 1.0, 1: 1.0, 2: 1.0})

In [56]:
#
from pyspark.ml.linalg import DenseVector
df = spark.createDataFrame([(DenseVector([1.0, 2.0]),),\
(DenseVector([0.0, 1.0]),), (DenseVector([3.0, 0.2]),)], ["tf"])
idf = IDF(minDocFreq=3, inputCol="tf", outputCol="idf")
model = idf.fit(df)
model.idf
model.transform(df).show()
idf.setParams(outputCol="freqs").fit(df).transform(df).collect()[1].freqs
params = {idf.minDocFreq: 1, idf.outputCol: "vector"}
idf.fit(df, params).transform(df).show()
# idfPath = temp_path + "/idf"
# idf.save(idfPath)
# loadedIdf = IDF.load(idfPath)
# loadedIdf.getMinDocFreq() == idf.getMinDocFreq()
# modelPath = temp_path + "/idf-model"
# model.save(modelPath)
# loadedModel = IDFModel.load(modelPath)
# loadedModel.transform(df).head().idf == model.transform(df).head().idf

+---------+---------+
|       tf|      idf|
+---------+---------+
|[1.0,2.0]|[0.0,0.0]|
|[0.0,1.0]|[0.0,0.0]|
|[3.0,0.2]|[0.0,0.0]|
+---------+---------+

+---------+--------------------+
|       tf|              vector|
+---------+--------------------+
|[1.0,2.0]|[0.28768207245178...|
|[0.0,1.0]|           [0.0,0.0]|
|[3.0,0.2]|[0.86304621735534...|
+---------+--------------------+



In [57]:
class pyspark.ml.feature.MaxAbsScaler(self, inputCol=None, outputCol=None)[source]¶
class pyspark.ml.feature.MaxAbsScalerModel(java_model=None)[source]¶
class pyspark.ml.feature.MinMaxScaler(self, min=0.0, max=1.0, inputCol=None, outputCol=None)[source]¶
class pyspark.ml.feature.NGram(self, n=2, inputCol=None, outputCol=None)[source]¶
class pyspark.ml.feature.Normalizer(self, p=2.0, inputCol=None, outputCol=None)[source]
class pyspark.ml.feature.OneHotEncoder(self, includeFirst=True, inputCol=None, outputCol=None)[source]
class pyspark.ml.feature.PCA(self, k=None, inputCol=None, outputCol=None)[source]¶
class pyspark.ml.feature.PolynomialExpansion(self, degree=2, inputCol=None, outputCol=None)[source]
class pyspark.ml.feature.QuantileDiscretizer(self, numBuckets=2, inputCol=None, outputCol=None, relativeError=0.001, 
class pyspark.ml.feature.RegexTokenizer(self, minTokenLength=1, gaps=True, pattern="s+", inputCol=None, outputCol=None, toLowercase=True)[source]¶
class pyspark.ml.feature.RFormula(self, formula=None, featuresCol="features", labelCol="label", forceIndexLabel=False)                                            
class pyspark.ml.feature.RFormulaModel(java_model=None)[source]
class pyspark.ml.feature.StandardScaler(self, withMean=False, withStd=True, inputCol=None, outputCol=None)[source]¶                                             
class pyspark.ml.feature.StopWordsRemover(self, inputCol=None, outputCol=None, stopWords=None, caseSensitive=false)[source]                        
class pyspark.ml.feature.StringIndexer(self, inputCol=None, outputCol=None, handleInvalid="error")[source]
class pyspark.ml.feature.Tokenizer(self, inputCol=None, outputCol=None)[source]
class pyspark.ml.feature.VectorAssembler(self, inputCols=None, outputCol=None)[source]¶
class pyspark.ml.feature.VectorIndexer(self, maxCategories=20, inputCol=None, outputCol=None)[source]¶
class pyspark.ml.feature.VectorSlicer(self, inputCol=None, outputCol=None, indices=None, names=None)[source]
class pyspark.ml.feature.Word2Vec(self, vectorSize=100, minCount=5, numPartitions=1, stepSize=0.025, maxIter=1, seed=None, inputCol=None, outputCol=None, windowSize=5, maxSentenceLength=1000)[source]
                                             

SyntaxError: invalid syntax (<ipython-input-57-8cfa3311a22b>, line 1)